# Batch producing

In [ ]:
# | hide

import platform

from IPython.display import Markdown as md

from fastkafka._components._subprocess import terminate_asyncio_process
from fastkafka._testing.apache_kafka_broker import run_and_match
from fastkafka.testing import ApacheKafkaBroker, run_script_and_cancel

If you want to send your data in batches `@produces` decorator makes that possible for you. By returning a `list` of messages you want to send in a batch the producer will collect the messages and send them in a batch to a Kafka broker.

This guide will demonstrate how to use this feature.

## Return a batch from the producing function

To define a batch that you want to produce to Kafka topic, you need to return the `List` of the messages that you want to be batched from your producing function.

In [ ]:
# | echo: false

hello_world_batch = """
from typing import List

@app.produces()
async def to_hello_world(msgs: List[str]) -> List[HelloWorld]:
    return [HelloWorld(msg=msg) for msg in msgs]
"""

md(f"```python\n{hello_world_batch}\n```")

```python

from typing import List

@app.produces()
async def to_hello_world(msgs: List[str]) -> List[HelloWorld]:
    return [HelloWorld(msg=msg) for msg in msgs]

```

In the example, we want to return the `HelloWorld` message class batch that is created from a list of msgs we passed into our producing function.

Lets also prepare a backgound task that will send a batch of "hello world" messages when the app starts.

In [ ]:
# | echo: false

bg_run = """
@app.run_in_background()
async def prepare_and_send_hello_batch():
    msgs=[f"Hello world {i}" for i in range(10)]
    await to_hello_world(msgs)
"""

md(f"```python\n{bg_run}\n```")

```python

@app.run_in_background()
async def prepare_and_send_hello_batch():
    msgs=[f"Hello world {i}" for i in range(10)]
    await to_hello_world(msgs)

```

## App example

We will modify the app example from [@producer basics](/docs/guides/Guide_21_Produces_Basics.md) guide to return the `HelloWorld` batch. The final app will look like this (make sure you replace the `<url_of_your_kafka_bootstrap_server>` and `<port_of_your_kafka_bootstrap_server>` with the actual values):

In [ ]:
# | hide

app = """
import asyncio
from fastkafka import FastKafka
from pydantic import BaseModel, Field

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)
"""

In [ ]:
# | echo: false

batch_example = app + bg_run + hello_world_batch

md(f"```python\n{batch_example}\n```")

```python

import asyncio
from fastkafka import FastKafka
from pydantic import BaseModel, Field

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)

@app.run_in_background()
async def prepare_and_send_hello_batch():
    msgs=[f"Hello world {i}" for i in range(10)]
    await to_hello_world(msgs)

from typing import List

@app.produces()
async def to_hello_world(msgs: List[str]) -> List[HelloWorld]:
    return [HelloWorld(msg=msg) for msg in msgs]

```

## Run the app

In [ ]:
# | echo: false

script_file = "producer_with_key_example.py"
cmd = "fastkafka run --num-workers=1 --kafka-broker=demo_broker producer_with_key_example:app"
md(
    f"Now we can run the app. Copy the code above in producer_example.py and run it by running\n```shell\n{cmd}\n```"
)

Now we can run the app. Copy the code above in producer_example.py and run it by running
```shell
fastkafka run --num-workers=1 --kafka-broker=demo_broker producer_with_key_example:app
```

After running the command, you should see this output in your terminal:

In [ ]:
# | hide

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    server_url = bootstrap_server.split(":")[0]
    server_port = bootstrap_server.split(":")[1]
    exit_code, output = await run_script_and_cancel(
        script=batch_example.replace(
            "<url_of_your_kafka_bootstrap_server>", server_url
        ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
        script_file=script_file,
        cmd=cmd,
        cancel_after=5,
    )

    expected_returncode = [0, 1]
    assert exit_code in expected_returncode, f'{exit_code=}, {output.decode("UTF-8")}'

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.sta

In [ ]:
# | echo: false

print(output.decode("UTF-8"))

[46480]: [INFO] fastkafka._application.app: run_in_background() : Adding function 'prepare_and_send_hello_batch' as background task
[46480]: [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:9092'
[46480]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9092'}'
[46480]: [INFO] fastkafka._application.app: _populate_bg_tasks() : Starting background task 'prepare_and_send_hello_batch'
Starting process cleanup, this may take a few seconds...
[INFO] fastkafka._server: terminate_asyncio_process(): Terminating the process 46480...
[46480]: [INFO] fastkafka._application.app: _shutdown_bg_tasks() : Cancelling background task 'prepare_and_send_hello_batch'
[46480]: [INFO] fastkafka._application.app: _shutdown_bg_tasks() : Waiting for background task 'prepare_and_send_hello_batch' to finish
[46480]: [INFO] fastkafka._application.app: _shutdown_bg_tasks() : Execution fin

## Check if the batch was sent to the Kafka topic with the defined key

Lets check the topic and see if there are "Hello world" messages in the hello_world topic. In your terminal run:

In [ ]:
# | echo: false

script_extension = ".bat" if platform.system() == "Windows" else ".sh"
consumer_cmd = f"kafka-console-consumer{script_extension} --topic=hello_world --from-beginning --bootstrap-server=<address_of_your_kafka_bootstrap_server>"
md(f"```shell\n{consumer_cmd}\n```")

```shell
kafka-console-consumer.sh --topic=hello_world --from-beginning --bootstrap-server=<address_of_your_kafka_bootstrap_server>
```

In [ ]:
# | hide

expected_msg = '{"msg": "Hello world *[0-9]"}'

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    server_url = bootstrap_server.split(":")[0]
    server_port = bootstrap_server.split(":")[1]
    exit_code, output = await run_script_and_cancel(
        script=batch_example.replace(
            "<url_of_your_kafka_bootstrap_server>", server_url
        ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
        script_file=script_file,
        cmd=cmd,
        cancel_after=5,
    )

    expected_returncode = [0, 1]
    assert exit_code in expected_returncode, f'{exit_code=}, {output.decode("UTF-8")}'

    proc = await run_and_match(
        *consumer_cmd.replace(
            "<address_of_your_kafka_bootstrap_server>", bootstrap_server
        ).split(" "),
        pattern=expected_msg,
        timeout=30,
        num_to_match=10
    )

    await terminate_asyncio_process(proc)

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] fastkafka._components._subprocess: terminate_asyncio_p

You should see the batch of messages in your topic.

## Batch key

To define a key for your batch like in [Defining a partition key](/docs/guides/Guide_22_Partition_Keys.md) guide you can wrap the returning value in a `KafkaEvent` class. To learn more about defining a partition ke and `KafkaEvent` class, please, have a look at [Defining a partition key](/docs/guides/Guide_22_Partition_Keys.md) guide.

Let's demonstrate that.

To define a key, we just need to modify our producing function, like this:

In [ ]:
# | echo: false

hello_world_batch_key = """
from typing import List
from fastkafka import KafkaEvent

@app.produces()
async def to_hello_world(msgs: List[str]) -> KafkaEvent[List[HelloWorld]]:
    return KafkaEvent([HelloWorld(msg=msg) for msg in msgs], key=b"my_key")
"""

md(f"```python\n{hello_world_batch_key}\n```")

```python

from typing import List
from fastkafka import KafkaEvent

@app.produces()
async def to_hello_world(msgs: List[str]) -> KafkaEvent[List[HelloWorld]]:
    return KafkaEvent([HelloWorld(msg=msg) for msg in msgs], key=b"my_key")

```

Now our app looks like this:

In [ ]:
# | echo: false

batch_key_example = app + bg_run + hello_world_batch_key

md(f"```python\n{batch_key_example}\n```")

```python

import asyncio
from fastkafka import FastKafka
from pydantic import BaseModel, Field

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)

@app.run_in_background()
async def prepare_and_send_hello_batch():
    msgs=[f"Hello world {i}" for i in range(10)]
    await to_hello_world(msgs)

from typing import List
from fastkafka import KafkaEvent

@app.produces()
async def to_hello_world(msgs: List[str]) -> KafkaEvent[List[HelloWorld]]:
    return KafkaEvent([HelloWorld(msg=msg) for msg in msgs], key=b"my_key")

```

## Check if the batch was sent to the Kafka topic

Lets check the topic and see if there are "Hello world" messages in the hello_world topic, containing a defined key. In your terminal run:

In [ ]:
# | echo: false

script_extension = ".bat" if platform.system() == "Windows" else ".sh"
consumer_cmd = f"kafka-console-consumer{script_extension} --topic=hello_world --property print.key=true --from-beginning --bootstrap-server=<address_of_your_kafka_bootstrap_server>"
md(f"```shell\n{consumer_cmd}\n```")

```shell
kafka-console-consumer.sh --topic=hello_world --property print.key=true --from-beginning --bootstrap-server=<address_of_your_kafka_bootstrap_server>
```

In [ ]:
# | hide

expected_msg = 'my_key	{"msg": "Hello world *[0-9]"}'

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    server_url = bootstrap_server.split(":")[0]
    server_port = bootstrap_server.split(":")[1]
    exit_code, output = await run_script_and_cancel(
        script=batch_key_example.replace(
            "<url_of_your_kafka_bootstrap_server>", server_url
        ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
        script_file=script_file,
        cmd=cmd,
        cancel_after=5,
    )

    expected_returncode = [0, 1]
    assert exit_code in expected_returncode, f'{exit_code=}, {output.decode("UTF-8")}'

    proc = await run_and_match(
        *consumer_cmd.replace(
            "<address_of_your_kafka_bootstrap_server>", bootstrap_server
        ).split(" "),
        pattern=expected_msg,
        timeout=30,
        num_to_match=1
    )

    await terminate_asyncio_process(proc)

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] fastkafka._components._subprocess: terminate_asyncio_p

You should see the batch of messages with the defined key in your topic.